In [105]:
from __future__ import print_function
from IPython.display import HTML, display

# 簡介
## 何時該用 Cython 來快速成型與加速


## 用 C/C++ 寫不好嗎？
* 一開始就就用 C/C++  ?
* 先用 Python，然後改寫成 C++ ?

有什麼理由要用 Cython ?


## 好的情形
[go.py](/edit/go/go.py) UCT go player in python, by mark.dufour@gmail.com.

利用 UCT 蒙地卡羅搜尋的圍棋 AI

### 測試跑的時間

|         |            時間 |
| ------------- |:--------------:|
| CPython2.7      | 11.6s |
| CPython3.4      |  13.1s|
| pypy| 8.92s|
| Cython| 598ms |

請參考 [Worksheet](/notebooks/Worksheet-go.ipynb)

Cython 的速度大約是 20 倍，完全一樣的程式碼，幾乎沒有修改

### 原因?
因為 cython 會吃 [go.pxd](/edit/go/go.pxd) 來使用型態定義，所以表面上是一樣的原始碼，其實 cython 偷偷多知道很多資訊。

# 一：加上型態
### 但要怎麼加？

-----------------------------------------------

## 不見得有用

[背包問題](https://zh.wikipedia.org/zh-tw/%E8%83%8C%E5%8C%85%E9%97%AE%E9%A2%98) DFS 演算法

### 常見的流程
* 一開始先用 python 弄一個很慢的原型 [knapsack_0_original.py](/edit/knapsack/knapsack_0_original.py)
* python 很好寫，但是很容易寫很慢(不過一開始慢不要緊，相讓邏輯正確)
* 查看一下問題出在哪裡
* 修改一個效率較好的版本 [knapsack_1_improved.py](/edit/knapsack/knapsack_1_improved.py)
* 像圍棋的例子一樣，加上型態試試看  [knapsack_2_typed.py](/edit/knapsack/knapsack_2_typed.py)

 
|                      | original| improved|typed |
| -------------      |:--------------:|:--------:|:------:|
| CPython2.7 | 8min 28s |31.7s  | 33.1s|
| CPython3.4 |  8min 12s|32.5s  |32.3s|
|pypy              |1min 16s |   4.01s|5.31s|
| Cython         | 7min 23s|30s      |31.3s|

## 加上型態一點用都沒有! why?

看看 knapsack 3

|                      | original| improved|typed |memoryview |
| -------------      |:--------------:|:--------:|:------:||
| CPython2.7 | 8min 28s |31.7s  | 33.1s|11.2s|
| CPython3.4 |  8min 12s|32.5s  |32.3s|7.4s|
|pypy              |1min 16s |   4.01s|5.31s|4.36s|
| Cython         | 7min 23s|30s      |31.3s|474ms|

## 這時連 python 的速度都增加了！

# 二: 用 C 的方式思考

### 能幫助寫出更快的程式

----------------------------------------------------------------

## Brainfuck interpreter

## mandelbrot.bf

|                      | dict|map |list|vector|vector/list|improved |inline|
| -------------    |:--------------:|:--------:|:------:|||
| CPython2.7 | ?| |?| | ?|1h 28min|1.54s|
|pypy              | ?||3min 35s| |3m 25s|1m 17s||
| Cython         | ?|?||1m 20s |1m 46s|29s/25s||

最快的 interpreter 約 6.5s，最快的編譯版本約 1.3s。

* 超級慢的原型 [bf0_original.py](/edit/brainfuck/bf0_original.py) (用 defaultdict)
    * [bf1_map.pyx](/edit/brainfuck/bf1_map.pyx) 用 stl map 替代 defaultdict
* [bf3_list_all.py](/edit/brainfuck/bf3_list_all.pyx)  因為記憶體使用上是連續的，用 python list 來取代 dict
    * [bf3_vector_all.pyx](/edit/brainfuck/bf3_vector_all.pyx)  用 stl vector 替代 python list
* 但是 vector 和 list 不一樣啊，程式碼不能共用，那乾脆直接寫 C++?
    * 可以共用 [bf4_merge.py](/edit/brainfuck/bf4_merge.py),  [bf4_merge.pxd](/edit/brainfuck/bf4_merge.pxd)
* 因為可以共用，繼續用 python 來嘗試新的可能。
    * [bf5_improved.py](/edit/brainfuck/bf5_improved.py), 嘗試一些最加速策略
    * [bf6_inline.py](/edit/brainfuck/bf6_inline.py), inline complied 版本

# 三：python 靈活性適合嘗試
### 加上速度，能有更多嘗試的機會